In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np  
import string
import os
from tensorflow.keras.optimizers import Adam 

# 1. Data Preparation

# 1a) Data Loading

with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()
# b) Text Cleaning
text = text.lower()
text = text.translate(str.maketrans('', '', string.punctuation))

# Create vocabulary and mapping
vocab = sorted(list(set(text)))
char_to_index = {char: index for index, char in enumerate(vocab)}
index_to_char = {index: char for index, char in enumerate(vocab)}

# c) Data Splitting (Simple 80/20 split)
train_size = int(len(text) * 0.8)
train_text = text[:train_size]
test_text = text[train_size:]

# d) Data Preprocessing

seq_length = 50

# Helper function to create sequences
def create_sequences(text):
    X = []
    y = []
    for i in range(0, len(text) - seq_length, 1):
        sequence = text[i:i + seq_length]
        target = text[i + seq_length]
        X.append([char_to_index[char] for char in sequence])
        y.append(char_to_index[target])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_text)
X_test, y_test = create_sequences(test_text)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=len(vocab))
y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(vocab))


# 2. Model Building

# a) RNN Architecture
model = tf.keras.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(seq_length, 1), return_sequences=True)) #Note: input_shape expects a 3D tensor (samples, timesteps, features)
model.add(keras.layers.LSTM(128))
# b) Output Layer
model.add(keras.layers.Dense(len(vocab), activation='softmax'))

# c) Model Compilation
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()

# 3. Training

# a) Training Process
model.fit(np.reshape(X_train, (X_train.shape[0], seq_length, 1)), y_train, epochs=2, batch_size=128, validation_data=(np.reshape(X_test, (X_test.shape[0], seq_length, 1)), y_test))

# 4. Evaluation


# a) Text Generation
def generate_text(length, seed_text):
  seed_text = seed_text.lower()
  seed_text = seed_text.translate(str.maketrans('', '', string.punctuation))
  pattern = [char_to_index[char] for char in seed_text]
  generated_text = seed_text
  for i in range(length):
    x = np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = index_to_char[index]
    generated_text += result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
  return generated_text

# Sample testing
seed_text = "the king"
generated_text = generate_text(20,seed_text)
print(f"Seed Text: {seed_text}")
print(f"Generated Text: {generated_text}")

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                        │ (None, 50, 128)             │          66,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 29)                  │           3,741 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 201,885 (788.61 KB)

 Trainable params: 201,885 (788.61 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/2
6631/6631 ━━━━━━━━━━━━━━━━━━━━ 74s 11ms/step - accuracy: 0.3347 - loss: 2.2540 - val_accuracy: 0.4239 - val_loss: 1.9232
Epoch 2/2
6631/6631 ━━━━━━━━━━━━━━━━━━━━ 70s 11ms/step - accuracy: 0.4753 - loss: 1.7279 - val_accuracy: 0.4457 - val_loss: 1.8544
Seed Text: the king
Generated Text: the king
moenu

soixer

nwor
